# Factorización de matrices
Los métodos implementados son los siguientes:
- Factorización LU: `factorizacion_lu`.
- Factorización de Cholesky: `factorizacion_cholesky`.
- Factorización QR: `factorizacion_qr`.

In [1]:
from numpy import *
from numpy.linalg import norm

## Factorización LU
Dada $A \in \mathcal{M}_n(\mathbb{C})$ inversible, se llama factorización LU a la descomposición, si es posible,
$$A = LU,$$
siendo $L \in \mathcal{M}_n(\mathbb{C})$ triangular inferior, inversible y con unos en la diagonal principal, y $U \in \mathcal{M}_n(\mathbb{C})$ triangular superior e inversible.

Si todas las submatrices principales de $A$ son también inversibles, entonces la factorización LU existe y es única.

In [2]:
def factorizacion_lu(A):
    m, n = shape(A)
    if m != n:
        print("La matriz no es cuadrada.")
        return None
    
    if A.dtype == complex:
        L = eye(n, dtype =  complex)
        U = array(A, dtype = complex)
    else:
        L = eye(n, dtype =  float)
        U = array(A, dtype = float)

    for k in range(n-1):
        if abs(A[k, k]) < 1e-15:
            print("No existe la factorización LU.")
            return None
        else:
            for i in range(k+1, n):
                L[i, k] = U[i, k]/U[k, k]
                U[i, k:] -= U[k, k:]*L[i, k]

    return L, U

Realizamos la factorización LU de la matriz
$$A = \begin{pmatrix}
2 & -1 & 4 & 0 \\
4 & -1 & 5 & 1 \\
-2 & 2 & -2 & 3 \\
0 & 3 & -9 & 4
\end{pmatrix}.$$

In [3]:
A = array([
    [2, -1, 4, 0],
    [4, -1, 5, 1],
    [-2, 2, -2, 3],
    [0, 3, -9, 4]
])

L, U = factorizacion_lu(A)
print("Matriz L\n", L)
print("Matriz U\n", U)

Matriz L
 [[ 1.  0.  0.  0.]
 [ 2.  1.  0.  0.]
 [-1.  1.  1.  0.]
 [ 0.  3.  0.  1.]]
Matriz U
 [[ 2. -1.  4.  0.]
 [ 0.  1. -3.  1.]
 [ 0.  0.  5.  2.]
 [ 0.  0.  0.  1.]]


## Factorización de Cholesky
Dada $A \in \mathcal{M}_n(\mathbb{C})$ hermítica e inversible, se llama factorización de Cholesky a la descomposición, si es posible,
$$A = CC^\ast,$$
siendo $C \in \mathcal{M}_n(\mathbb{C})$ triangular inferior e inversible.

Si $A$ es definida positiva, entonces la factorización de Cholesky existe.
Además, si se impone que todos los elementos diagonales de la matriz $C$ sean positivos, entonces la factorización es única.

In [4]:
def factorizacion_cholesky(A):
    m, n = shape(A)
    if m != n:
        print("La matriz no es cuadrada.")
        return None

    if A.dtype == complex:
        C = zeros((n, n), complex)
    else:
        C = zeros((n, n), float)

    for i in range(n):
        C[i, i] = sqrt(A[i, i] - sum(abs(C[i, :i])**2))
        if abs(C[i, i]) < 1e-15:
            print("No existe la factorización de Cholesky.")
            return None
        else:
            for j in range(i+1, n):
                C[j, i] = (A[i, j] - sum(C[i, :i]*C[j, :i]))/C[i, i]
    
    return C

Realizamos la factorización de Cholesky de la matriz
$$ A = \begin{pmatrix}
1 & 2 & 3 & 4 \\
2 & 5 & 1 & 10 \\
3 & 1 & 35 & 5 \\
4 & 10 & 5 & 45
\end{pmatrix}.$$

In [5]:
A = array([
    [1, 2, 3, 4],
    [2, 5, 1, 10],
    [3, 1, 35, 5],
    [4, 10, 5, 45]
])

C = factorizacion_cholesky(A)
print("Matriz C\n", C)

Matriz C
 [[ 1.  0.  0.  0.]
 [ 2.  1.  0.  0.]
 [ 3. -5.  1.  0.]
 [ 4.  2.  3.  4.]]


## Factorización QR
Dada $A \in \mathcal{M}_n(\mathbb{C})$ inversible, se llama factorización QR a la descomposición, si es posible,
$$A = QR,$$
donde $Q \in \mathcal{M}_n(\mathbb{C})$ es unitaria y $R \in \mathcal{M}_n(\mathbb{C})$ es triangular superior e inversible.

Si la matriz $A$ es inversible, entonces la factorización QR existe.
Además, si se impone que todos los elementos diagonales de $R$ sean positivos, entonces la factorización es única.

In [6]:
def factorizacion_qr(A):
    m, n = shape(A)
    if m != n:
        print("La matriz no es cuadrada.")
        return None

    if A.dtype == complex:
        Q = eye(n, dtype = complex)
        R = array(A, dtype = complex)
    else:
        Q = eye(n, dtype = float)
        R = array(A, dtype = float)

    for k in range(n-1):
        wk = copy(R[k:, k])
        chi = norm(wk, 2)
        if A.dtype == complex:
            wk[0] += chi*exp(angle(wk[0])*1j)
            vk = zeros((n, 1), dtype = complex)
        else:
            wk[0] += chi*sign(wk[0]) + chi*(wk[0] == 0)
            vk = zeros((n, 1), dtype = float)
        vk[k:, 0] = wk[:]
        Hk = eye(n) - 2*(vk@vk.conjugate().T)/(vk.conjugate().T@vk)
        Q = Q@transpose(Hk)
        R = Hk@R
        
    return Q, R

Realizamos la factorización QR de la matriz
$$ A = \begin{pmatrix}
12 & -51 & 4 \\
6 & 167 & -68 \\
-4 & 24 & -41
\end{pmatrix}.$$

In [7]:
A = array([
    [12, -51, 4],
    [6, 167, -68],
    [-4, 24, -41]
])

Q, R = factorizacion_qr(A)
print("Matriz Q\n", Q)
print("Matriz R\n", R)

Matriz Q
 [[-0.85714286  0.39428571  0.33142857]
 [-0.42857143 -0.90285714 -0.03428571]
 [ 0.28571429 -0.17142857  0.94285714]]
Matriz R
 [[-1.40000000e+01 -2.10000000e+01  1.40000000e+01]
 [-5.57673565e-16 -1.75000000e+02  7.00000000e+01]
 [-5.08994556e-16 -7.64989650e-16 -3.50000000e+01]]
